In [1]:
from bs4 import BeautifulSoup
import requests
import os
import re
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download("stopwords")
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from xgboost import XGBClassifier



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\floPe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\floPe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def find_hashtags(tweet):
    return " ".join([match.group(0)[1:] for match in re.finditer(r"#\w+", tweet)]) or 'no'
def find_mentions(tweet):
    return " ".join([match.group(0)[1:] for match in re.finditer(r"@\w+", tweet)]) or 'no'
def find_links(tweet):
    return " ".join([match.group(0)[:] for match in re.finditer(r"https?://\S+", tweet)]) or 'no'


In [23]:
def tolkencleaner(text):
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    ps = PorterStemmer()
    tolkenized = []
    no_link = re.sub(r'https?://\S+', '', text)
    no_break = re.sub(r'\n',' ', no_link)
    cleaner = re.sub('\s+', ' ', no_break).strip()
    word_tokens = word_tokenize(cleaner)
    stripped = [ps.stem(w) for w in word_tokens if not w in punctuation]
    split_sentence = [w for w in stripped if not w in stop_words]
    filtered_sentence = [w.lower() for w in split_sentence]
    finished_sentence = ' '.join(filtered_sentence)
    tolkenized.append(finished_sentence)
    df = pd.DataFrame(tolkenized, columns=['text'])
    df['hashtag'] = df['text'].apply(lambda x: find_hashtags(x))
    df['mentions'] = df['text'].apply(lambda x: find_mentions(x))
    df['links'] = df['text'].apply(lambda x: find_links(x))
    print(df)
    return df


    

In [40]:


def vectorizing(df):
    cv_hashtag = CountVectorizer()
    cv_mentions = CountVectorizer()
#     cv_link = CountVectorizer(analyzer = 'word', token_pattern = r'https?://\S+')
    ht = cv_hashtag.fit_transform(df['hashtag'])
    mt = cv_mentions.fit_transform(df['mentions'])
#     lt = cv_link.fit_transform(df['links'])
    print("look herererererererrer", ht, mt)
    train_ht = pd.DataFrame(ht.toarray(), columns=cv_hashtag.get_feature_names())
    train_mt = pd.DataFrame(mt.toarray(), columns=cv_mentions.get_feature_names())
    #     train_lt = pd.DataFrame(lt.toarray(), columns=cv_link.get_feature_names())
    print("look herererererererrer", train_ht, train_mt)
    vec_text = TfidfVectorizer(ngram_range = (1,2)) 
    vt = vec_text.fit_transform(df['text'])
 
    print("look",vt)
    train_txt = pd.DataFrame(vt.toarray(), columns=vec_text.get_feature_names())


    T = pd.DataFrame(train_txt)
#     T = T.join(train_lt, rsuffix='_link')
    T = T.join(train_ht, rsuffix='_hashtags')
    T = T.join(train_mt, rsuffix='_mentions')
    print(T)
    return T

In [49]:
def prediction(g):
    # mars_data
    # vectorizing = vectorizing()
    # tolkencleaner = tolkencleaner()
    clf = pickle.load(open('finalized_model2.sav', 'rb'))
     
    label = {0:'negative', 1:'positive'}
    d = tolkencleaner(g)
    X = vectorizing(d)
    print('Prediction: %s\nProbability: %.2f%%' %\
          (label[clf.predict(X)[0]],
           np.max(clf.predict_proba(X))*100))
    tweets = (label[clf.predict(X)[0]])
    print("lastly" , tweets)    
   


      
    p = pd.DataFrame(tweets)

    
    return p

In [48]:
g = "Morty, get your shit together fire god one "
prediction(g)

                                 text hashtag mentions links
0  morti get shit togeth fire god one      no       no    no
look herererererererrer   (0, 0)	1   (0, 0)	1
look herererererererrer    no
0   1    no
0   1
look   (0, 5)	0.2773500981126146
  (0, 1)	0.2773500981126146
  (0, 12)	0.2773500981126146
  (0, 10)	0.2773500981126146
  (0, 3)	0.2773500981126146
  (0, 7)	0.2773500981126146
  (0, 8)	0.2773500981126146
  (0, 4)	0.2773500981126146
  (0, 0)	0.2773500981126146
  (0, 11)	0.2773500981126146
  (0, 9)	0.2773500981126146
  (0, 2)	0.2773500981126146
  (0, 6)	0.2773500981126146
      fire  fire god      get  get shit      god  god one    morti  morti get  \
0  0.27735   0.27735  0.27735   0.27735  0.27735  0.27735  0.27735    0.27735   

       one     shit  shit togeth   togeth  togeth fire  no  no_mentions  
0  0.27735  0.27735      0.27735  0.27735      0.27735   1            1  


ValueError: feature_names mismatch: ['00', '01', '04', '05', '06', '07', '08', '08 05', '08 06', '10', '100', '11', '11 year', '12', '12000', '12000 nigerian', '13', '130', '130 special', '14', '15', '15 saudi', '16', '16yr', '16yr old', '17', '18', '1980', '1st', '20', '2013', '2014', '2015', '2015 prebreak', '22', '24', '25', '29', '2nd', '30', '40', '40 famili', '50', '5km', '5km volcano', '60', '600', '70', '70 year', '70th', '70th anniversari', '90', 'aba', 'aba woman', 'abandon', 'abc', 'abc new', 'abl', 'ablaz', 'absolut', 'accid', 'accord', 'account', 'act', 'action', 'activ', 'actual', 'ad', 'ad video', 'affect', 'affect fatal', 'africa', 'aftershock', 'ago', 'agre', 'ahead', 'ai', 'air', 'air ambul', 'aircraft', 'aircraft debri', 'airlin', 'airplan', 'airplan accid', 'airport', 'airport swallow', 'alabama', 'alabama home', 'alarm', 'album', 'aliv', 'allow', 'allow parol', 'alon', 'alreadi', 'alway', 'amazon', 'ambul', 'ambul helicopt', 'america', 'american', 'amid', 'amid cri